### PyCitySchools

In [1]:
# Setup: Import resources and modules
import pandas as pd
import numpy as np 

school_data_to_load = "Resources/school_info.csv"
student_data_to_load = "Resources/student_info.csv"

# Read data 
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
merged_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [52]:
# Count number of schools 
total_schools = len(merged_df["school_name"].unique())

# Sum budget from school_data file 
total_budget = school_data["budget"].sum()

# Collect numeric value summaries and pull data 
num_summary = merged_df.describe()
total_students = num_summary.loc["count","School ID"]
avg_math = num_summary.loc["mean", "math_score"]
avg_read = num_summary.loc["mean", "reading_score"]

# Calculate passing scores 
math_pass = merged_df.loc[merged_df["math_score"] >= 70]
per_math = len(math_pass) / total_students * 100

read_pass = merged_df.loc[merged_df["reading_score"] >= 70]
per_reading = len(read_pass) / total_students * 100

math_reading = merged_df.loc[(merged_df["math_score"]>=70) & (merged_df["reading_score"]>=70)]
math_reading = len(math_reading) / total_students * 100

#Round values
avg_math = avg_math.round(1)
avg_read = avg_read.round(1)

# Create summary dataframe 
district_summary = {"Total Schools": total_schools, 
                    "Total Students": total_students,
                    "Total Budget": total_budget,  
                    "Average Math Score": avg_math, 
                    "Average Reading Score": avg_read,
                   "Percent Passing Math": per_math,
                   "Percent Passing Reading": per_reading,
                   "Percent Passing Math & Reading": math_reading}
district_summary_df = pd.DataFrame([district_summary])

# Formatting
district_summary_df["Percent Passing Math"] = district_summary_df["Percent Passing Math"].map("{:.1f}%".format)
district_summary_df["Percent Passing Reading"] = district_summary_df["Percent Passing Reading"].map("{:.1f}%".format)
district_summary_df["Percent Passing Math & Reading"] = district_summary_df["Percent Passing Math & Reading"].map("{:.1f}%".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.0f}".format)
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,.0f}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Math & Reading
0,15,"39,170","$24,649,428",79.0,81.9,75.0%,85.8%,65.2%


In [53]:
## Calculations 

# Re-index by school
schools_by_name = merged_df.set_index("school_name")
total_students = schools_by_name.groupby("school_name")["Student ID"].count()

# Select passing scores 
avg_math = schools_by_name.groupby('school_name')["math_score"].mean()
avg_reading = schools_by_name.groupby('school_name')["reading_score"].mean()

# Select all passing scores 
passing_math = schools_by_name.loc[schools_by_name["math_score"]>=70]
passing_reading = schools_by_name.loc[schools_by_name["reading_score"]>=70]

# Group scores by school and count number of passing grades
num_passing_math = passing_math.groupby('school_name')['math_score'].count()
num_passing_reading = passing_reading.groupby('school_name')['reading_score'].count()
percent_passing_math = num_passing_math / total_students * 100
percent_passing_reading = num_passing_reading / total_students * 100 
 
# Select scores that pass both math & reading
math_reading = schools_by_name.loc[(schools_by_name["math_score"]>=70) & (schools_by_name["reading_score"]>=70)]
#Count by school
num_overall = math_reading.groupby("school_name")["grade"].count()
overall = num_overall / total_students * 100

summary_df = {"Avg Math Score": avg_math,
              "Avg Reading Score": avg_reading,
            "Percent Passing Math": percent_passing_math, 
             "Percent Passing Reading": percent_passing_reading, 
             "Percent Passing Math & Reading": overall}
summary_df = pd.DataFrame(summary_df)

# Get existing info from school data file and merge with calculations 
columns = ["school_name", "type", "size", "budget"]
schools_df = school_data[columns]

# Add per student budget column 
budget = schools_df["budget"]
students = schools_df["size"]
per_student = budget / students 
schools_df["Per Student Budget"] = per_student

# Merge dataframes
schools_df = pd.merge(schools_df, summary_df, on="school_name")

# Save unformatted DF for later 
schools_df_unformatted = schools_df

# Formatting
schools_df = schools_df.rename(columns={"school_name": "School Name",
                                       "type": "Type",
                                       "size": "Total Students",
                                       "budget": "Budget"})
schools_df["Budget"] = schools_df["Budget"].map("${:,.0f}".format)
schools_df["Total Students"] = schools_df["Total Students"].map("{:,.0f}".format)
schools_df["Per Student Budget"] = schools_df["Per Student Budget"].map("${:,.0f}".format)
schools_df["Avg Math Score"] = schools_df["Avg Math Score"].map("{:.1f}".format)
schools_df["Avg Reading Score"] = schools_df["Avg Reading Score"].map("{:.1f}".format)
schools_df["Percent Passing Reading"] = schools_df["Percent Passing Reading"].map("{:.1f}%".format)
schools_df["Percent Passing Math"] = schools_df["Percent Passing Math"].map("{:.1f}%".format)
schools_df["Percent Passing Math & Reading"] = schools_df["Percent Passing Math & Reading"].map("{:.1f}%".format)

schools_df

,School Name,Type,Total Students,Budget,Per Student Budget,Avg Math Score,Avg Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Math & Reading
0,Huang High School,District,"2,917","$1,910,635",$655,76.6,81.2,65.7%,81.3%,53.5%
1,Figueroa High School,District,"2,949","$1,884,411",$639,76.7,81.2,66.0%,80.7%,53.2%
2,Shelton High School,Charter,"1,761","$1,056,600",$600,83.4,83.7,93.9%,95.9%,89.9%
3,Hernandez High School,District,"4,635","$3,022,020",$652,77.3,80.9,66.8%,80.9%,53.5%
4,Griffin High School,Charter,"1,468","$917,500",$625,83.4,83.8,93.4%,97.1%,90.6%
5,Wilson High School,Charter,"2,283","$1,319,574",$578,83.3,84.0,93.9%,96.5%,90.6%
6,Cabrera High School,Charter,"1,858","$1,081,356",$582,83.1,84.0,94.1%,97.0%,91.3%
7,Bailey High School,District,"4,976","$3,124,928",$628,77.0,81.0,66.7%,81.9%,54.6%
8,Holden High School,Charter,427,"$248,087",$581,83.8,83.8,92.5%,96.3%,89.2%
9,Pena High School,Charter,962,"$585,858",$609,83.8,84.0,94.6%,95.9%,90.5%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [105]:
# Sort by Percent Passing Overall and show top 5
overall_performance = schools_df_unformatted.sort_values("Percent Passing Math & Reading", ascending = False)


# Rename 
overall_performance = overall_performance.rename(columns={"school_name": "School Name",
                                               "type": "Type", 
                                               "size": "Total Students",
                                               "budget": "Budget"})
# Formatting 
overall_performance["Budget"] = overall_performance["Budget"].map("${:,.0f}".format)
overall_performance["Total Students"] = overall_performance["Total Students"].map("{:,.0f}".format)
overall_performance["Per Student Budget"] = overall_performance["Per Student Budget"].map("${:,.0f}".format)
overall_performance["Avg Math Score"] = overall_performance["Avg Math Score"].map("{:.1f}".format)
overall_performance["Avg Reading Score"] = overall_performance["Avg Reading Score"].map("{:.1f}".format)
overall_performance["Percent Passing Math"] = overall_performance["Percent Passing Math"].map("{:.1f}%".format)
overall_performance["Percent Passing Reading"] = overall_performance["Percent Passing Reading"].map("{:.1f}%".format)
overall_performance["Percent Passing Math & Reading"] = overall_performance["Percent Passing Math & Reading"].map("{:.1f}%".format)

top_performing_overall = overall_performance.head(5)
top_performing_overall

,School Name,Type,Total Students,Budget,Per Student Budget,Avg Math Score,Avg Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Math & Reading
6,Cabrera High School,Charter,"1,858","$1,081,356",$582,83.1,84.0,94.1%,97.0%,91.3%
14,Thomas High School,Charter,"1,635","$1,043,130",$638,83.4,83.8,93.3%,97.3%,90.9%
4,Griffin High School,Charter,"1,468","$917,500",$625,83.4,83.8,93.4%,97.1%,90.6%
5,Wilson High School,Charter,"2,283","$1,319,574",$578,83.3,84.0,93.9%,96.5%,90.6%
9,Pena High School,Charter,962,"$585,858",$609,83.8,84.0,94.6%,95.9%,90.5%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [108]:
bottom_performing = overall_performance.tail(5)
bottom_performing

,School Name,Type,Total Students,Budget,Per Student Budget,Avg Math Score,Avg Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Math & Reading
12,Johnson High School,District,"4,761","$3,094,650",$650,77.1,81.0,66.1%,81.2%,53.5%
3,Hernandez High School,District,"4,635","$3,022,020",$652,77.3,80.9,66.8%,80.9%,53.5%
0,Huang High School,District,"2,917","$1,910,635",$655,76.6,81.2,65.7%,81.3%,53.5%
1,Figueroa High School,District,"2,949","$1,884,411",$639,76.7,81.2,66.0%,80.7%,53.2%
11,Rodriguez High School,District,"3,999","$2,547,363",$637,76.8,80.7,66.4%,80.2%,53.0%


## Reading Scores by Grade

In [6]:
# Select all data for 9th grade and find mean for reading scores 
ninth = student_data.loc[student_data["grade"] == "9th"]
ninth = ninth.groupby("school_name")["reading_score"].mean()

# Select all data for 10th grade and find mean for reading scores 
tenth = student_data.loc[student_data["grade"] == "10th"]
tenth = tenth.groupby("school_name")["reading_score"].mean()

# Select all data for 11th grade and find mean for reading scores 
eleventh = student_data.loc[student_data["grade"] == "11th"]
eleventh = eleventh.groupby("school_name")["reading_score"].mean()

# Select all data for 12th grade and find mean for reading scores 
twelfth = student_data.loc[student_data["grade"] == "12th"]
twelfth = twelfth.groupby("school_name")["reading_score"].mean()

# Add to dataframe 
reading_scores = {"9th Grade": ninth, 
                 "10th Grade": tenth, 
                 "11th Grade": eleventh, 
                 "12th Grade": twelfth}
r_scores = pd.DataFrame(reading_scores)

#Reset Index as a column and change header name  
r_scores = r_scores.reset_index()
r_scores = r_scores.rename(columns={"school_name": "School Name"})


# Formatting 
r_scores["9th Grade"] = r_scores["9th Grade"].map("{:.1f}".format)
r_scores["10th Grade"] = r_scores["10th Grade"].map("{:.1f}".format)
r_scores["11th Grade"] = r_scores["11th Grade"].map("{:.1f}".format)
r_scores["12th Grade"] = r_scores["12th Grade"].map("{:.1f}".format)

r_scores

,School Name,9th Grade,10th Grade,11th Grade,12th Grade
0,Bailey High School,81.3,80.9,80.9,80.9
1,Cabrera High School,83.7,84.3,83.8,84.3
2,Figueroa High School,81.2,81.4,80.6,81.4
3,Ford High School,80.6,81.3,80.4,80.7
4,Griffin High School,83.4,83.7,84.3,84.0
5,Hernandez High School,80.9,80.7,81.4,80.9
6,Holden High School,83.7,83.3,83.8,84.7
7,Huang High School,81.3,81.5,81.4,80.3
8,Johnson High School,81.3,80.8,80.6,81.2
9,Pena High School,83.8,83.6,84.3,84.6


## Math Scores by Grade 

In [7]:
# Select all data for 9th grade and find mean for reading scores 
ninth_grade = student_data.loc[student_data["grade"] == "9th"]
ninth_grade = ninth_grade.groupby("school_name")["math_score"].mean()

# Select all data for 10th grade and find mean for reading scores 
tenth_grade = student_data.loc[student_data["grade"] == "10th"]
tenth_grade = tenth_grade.groupby("school_name")["math_score"].mean()

# Select all data for 11th grade and find mean for reading scores 
eleventh_grade = student_data.loc[student_data["grade"] == "11th"]
eleventh_grade = eleventh_grade.groupby("school_name")["math_score"].mean()

# Select all data for 12th grade and find mean for reading scores 
twelfth_grade = student_data.loc[student_data["grade"] == "12th"]
twelfth_grade = twelfth_grade.groupby("school_name")["math_score"].mean()

# Add to dataframe 
math_scores = {"9th Grade": ninth_grade, 
                 "10th Grade": tenth_grade, 
                 "11th Grade": eleventh_grade, 
                 "12th Grade": twelfth_grade}
m_scores = pd.DataFrame(math_scores)

#Reset Index as a column and change header name  
m_scores = m_scores.reset_index()
m_scores = m_scores.rename(columns={"school_name": "School Name"})


# Formatting 
m_scores["9th Grade"] = m_scores["9th Grade"].map("{:.1f}".format)
m_scores["10th Grade"] = m_scores["10th Grade"].map("{:.1f}".format)
m_scores["11th Grade"] = m_scores["11th Grade"].map("{:.1f}".format)
m_scores["12th Grade"] = m_scores["12th Grade"].map("{:.1f}".format)

m_scores

,School Name,9th Grade,10th Grade,11th Grade,12th Grade
0,Bailey High School,77.1,77.0,77.5,76.5
1,Cabrera High School,83.1,83.2,82.8,83.3
2,Figueroa High School,76.4,76.5,76.9,77.2
3,Ford High School,77.4,77.7,76.9,76.2
4,Griffin High School,82.0,84.2,83.8,83.4
5,Hernandez High School,77.4,77.3,77.1,77.2
6,Holden High School,83.8,83.4,85.0,82.9
7,Huang High School,77.0,75.9,76.4,77.2
8,Johnson High School,77.2,76.7,77.5,76.9
9,Pena High School,83.6,83.4,84.3,84.1


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [97]:
# Select columns
summary_columns = ["Avg Math Score", 
                   "Avg Reading Score", "Percent Passing Math", 
                   "Percent Passing Reading", "Percent Passing Math & Reading"]
spending_summary = schools_df_unformatted[summary_columns]

# Create data frame with numerical value for budget per student 
budget = {"PS Budget": per_student}
budget_df = pd.DataFrame(budget)

# Create bins for spending ranges and respective labels 
spending_limits = [550, 599, 649, 700]
spending_ranges = ["550 - 599", "600 - 649", "650 - 699"]
budget_df["Spending Range"] = pd.cut(budget_df["PS Budget"], spending_limits, labels=spending_ranges, include_lowest=False)

# Add column to the summary chart 
spending_summary = pd.merge(spending_summary, budget_bins, left_index=True, right_index=True)

# Format spending range column 
spending_summary["Spending Range"] = spending_summary["Spending Range"].map("${}".format)

# Calculate averages by spending range
spend_math = spending_summary.groupby('Spending Range')["Avg Math Score"].mean()
spend_read = spending_summary.groupby('Spending Range')["Avg Reading Score"].mean()
spend_overall = spending_summary.groupby('Spending Range')["Percent Passing Math & Reading"].mean()
spend_math_pc = spending_summary.groupby('Spending Range')["Percent Passing Math"].mean()
spend_read_pc = spending_summary.groupby('Spending Range')["Percent Passing Reading"].mean()

# Create dataframe 
spend_list = {"Avg Math Score": spend_math,
             "Avg Reading Score": spend_read,
             "% Passing Math": spend_math,
             "% Passing Reading": spend_read,
             "% Passing Overall": spend_overall}
spend_df = pd.DataFrame(spend_list)

# Formatting 
spend_df["% Passing Math"] = spend_df["% Passing Math"].map("{:.1f}%".format)
spend_df["% Passing Reading"] = spend_df["% Passing Reading"].map("{:.1f}%".format)
spend_df["% Passing Overall"] = spend_df["% Passing Overall"].map("{:.1f}%".format)
spend_df["Avg Math Score"] = spend_df["Avg Math Score"].map("{:.1f}%".format)
spend_df["Avg Reading Score"] = spend_df["Avg Reading Score"].map("{:.1f}%".format)

spend_df

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Spending Range,,,,,
$550 - 599,83.5%,83.9%,83.5%,83.9%,90.4%
$600 - 649,80.2%,82.4%,80.2%,82.4%,72.1%
$650 - 699,77.0%,81.0%,77.0%,81.0%,53.5%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [85]:
# Select columns 
sum_columns = ["Avg Math Score", 
                   "Avg Reading Score", "Percent Passing Math", 
                   "Percent Passing Reading", "Percent Passing Math & Reading", "size"]
school_size = schools_df_unformatted[sum_columns]

# Avoid copy df warning 
school_size = pd.DataFrame(school_size)

# Create bins and cut the dataframe
student_bins = [400, 999, 1999, 2999, 3999, 4999]
student_labels = ["400 to 999", "1k to 1999", "2k to 2999", "3k to 3999", "4k to 5k"]
school_size["Total Students"] = pd.cut(school_size["size"], student_bins, labels=student_labels, include_lowest=False)

# Get summary based on total students      
size_overall = school_size.groupby('Total Students')["Percent Passing Math & Reading"].mean()
size_math_avg = school_size.groupby('Total Students')["Avg Math Score"].mean()
size_reading_avg = school_size.groupby('Total Students')["Avg Reading Score"].mean()
size_math_pc = school_size.groupby('Total Students')["Percent Passing Math"].mean()
size_reading_pc = school_size.groupby('Total Students')["Percent Passing Reading"].mean()

# Create dataframe 
size_list = {"Avg Math Score":size_math_avg, 
          "Avg Reading Score": size_reading_avg, 
          "% Passing Math": size_math_pc, 
          "% Passing Reading": size_reading_pc,
          "% Passing Overall": size_overall}
size_df = pd.DataFrame(size_list)


# Formatting 
size_df["% Passing Math"] = size_df["% Passing Math"].map("{:.1f}%".format)
size_df["% Passing Reading"] = size_df["% Passing Reading"].map("{:.1f}%".format)
size_df["% Passing Overall"] = size_df["% Passing Overall"].map("{:.1f}%".format)
size_df["Avg Math Score"] = size_df["Avg Math Score"].map("{:.1f}".format)
size_df["Avg Reading Score"] = size_df["Avg Reading Score"].map("{:.1f}".format)


# Sort by size 
size_df

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Total Students,,,,,
400 to 999,83.8,83.9,93.6%,96.1%,89.9%
1k to 1999,83.4,83.9,93.6%,96.8%,90.6%
2k to 2999,78.4,81.8,73.5%,84.5%,62.9%
3k to 3999,76.8,80.7,66.4%,80.2%,53.0%
4k to 5k,77.1,81.0,66.5%,81.3%,53.9%


## Scores by School Type

* Perform the same operations as above, based on school type

In [68]:
# Select columns
type_columns = ["school_name", "Avg Math Score", 
                   "Avg Reading Score", "Percent Passing Math", 
                   "Percent Passing Reading", "Percent Passing Math & Reading", "type"]
type_summary = schools_df_unformatted[type_columns]

# Find average scores grouped by school type 
type_summary_avg = type_summary.groupby('type')["Percent Passing Math & Reading"].mean()
type_pass_math = type_summary.groupby('type')["Percent Passing Reading"].mean()
type_pass_reading = type_summary.groupby('type')["Percent Passing Math"].mean()
type_math_avg = type_summary.groupby('type')["Avg Math Score"].mean()
type_reading_avg = type_summary.groupby('type')["Avg Reading Score"].mean()

# Create summary df 
type_list = {"Avg Math Score": type_math_avg, 
          "Avg Reading Score": type_reading_avg, 
          "% Passing Math": type_pass_math, 
          "% Passing Reading": type_pass_reading,
          "% Passing Overall": type_summary_avg}
type_df = pd.DataFrame(type_list)

# Formatting 
type_df = type_df.rename_axis(None, axis=1).rename_axis('Type', axis=0)
type_df["Avg Math Score"] = type_df["Avg Math Score"].map("{:.1f}".format)
type_df["Avg Reading Score"] = type_df["Avg Reading Score"].map("{:.1f}".format)
type_df["% Passing Math"] = type_df["% Passing Math"].map("{:.1f}%".format)
type_df["% Passing Reading"] = type_df["% Passing Reading"].map("{:.1f}%".format)
type_df["% Passing Overall"] = type_df["% Passing Overall"].map("{:.1f}%".format)
type_df

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Type,,,,,
Charter,83.5,83.9,96.6%,93.6%,90.4%
District,77.0,81.0,80.8%,66.5%,53.7%
